In [ ]:
#import modules for analyzing ,plotting, and formatting
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns


In [ ]:
merged_400_1600 = pd.read_csv("merged_400m_1600m_df.csv")

In [ ]:
#import sklearn and relevant packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

#separate datasets for the model
#here the model is split 67/33
m400_1600_train, m400_1600_test = train_test_split(merged_400_1600, test_size=0.35, random_state=10)

#gl means grade level
#for this model we will use 400/1600m times and grade level
gl_400_1600_train = m400_1600_train[['400 Meters', '1600 Meters', 'Grade Level']]
gl_400_1600_test = m400_1600_test[['400 Meters', '1600 Meters', 'Grade Level']]


In [ ]:
#here we separate the features and labels. The features used are the 400m times as well as the grade leve
# this calls for dropping the 1600m column for the features df
#next I select only the 1600m column for the label df
features_400_train = gl_400_1600_train.drop('1600 Meters', axis=1)
label_train = gl_400_1600_train['1600 Meters'].copy()

#same steps are repeated for the test df
features_400_test = gl_400_1600_test.drop('1600 Meters', axis=1)
label_test = gl_400_1600_test['1600 Meters'].copy()

In [ ]:
f = features_400_train[features_400_train['Grade Level'] == '9th Grade']
f

,400 Meters,Grade Level
45,63.74,9th Grade
18,63.24,9th Grade
12,56.80,9th Grade
46,72.50,9th Grade
30,56.74,9th Grade
64,66.74,9th Grade


In [ ]:
#regression models require numeric data, this means that we have to convert variables from a categorical feature to a binary one.
#grade level is converted to 0s and 1s
# get dummies turned all non-numeric to numeric.
dummy_400_train = pd.get_dummies(features_400_train)

#after converting the grade level column, we reindex the df
dummy_400_train = dummy_400_train.reindex(columns=dummy_400_train.columns, fill_value=0)
features_400_train = dummy_400_train[['400 Meters', 'Grade Level_9th Grade', 'Grade Level_10th Grade', 'Grade Level_11th Grade', 'Grade Level_12th Grade']]
features_400_train

#the same steps are repeated for the test df
dummy_400_test = pd.get_dummies(features_400_test)

dummy_400_test = dummy_400_test.reindex(columns=dummy_400_test.columns, fill_value=0)
features_400_test = dummy_400_test[['400 Meters', 'Grade Level_9th Grade' ,'Grade Level_10th Grade', 'Grade Level_11th Grade', 'Grade Level_12th Grade']]


In [ ]:
#here I am selecting the runners with times between 59 and 60 seconds in the 400m
#next I am finding those same runners in the labeled data (1600m times)
#I do this as a simple measure so that I can compare the single test predicitions below, and get 
# a sense of which model is predicting closes to what we expect a runner to run. 

times = features_400_train[(features_400_train['400 Meters'] > 59) & (features_400_train['400 Meters'] < 60)]
label_train[times.index].mean()

322.01000000000005

In [ ]:
# I used a small definition from Intro to machine learning to display the scores from the cross val.
#this code will be used later to display the scores from cross val cv
def display_scores(scores):
  print('Scores:', scores)
  print('Mean:', scores.mean()),
  print('Standard Deviation:', scores.std())


Here we have the data processed and formatted to run multiple ML models.
These will be LinearRegression, Decision Tree Regression, and Forest Regression.

Our main measure of the data is the RMSE. This function compares the values of the labels to the predictions. We compare this value for all the training data and choose a model that works best before we use the test data.

In [ ]:
#after preparing the data we are ready to create the linear regression instance
# and train the model with out data

linreg = LinearRegression()
linreg.fit(features_400_train, label_train)

#here we are importing another regression model to test the difference,
# and checking is a more powerful model will produce better results.
#we fit this model and predict on our training data as we did before.

tree_reg = DecisionTreeRegressor()
tree_reg.fit(features_400_train, label_train)

#the most powerful model used in this project
forest_reg = RandomForestRegressor()
forest_reg.fit(features_400_train, label_train)

#we import the one more regression model which will be the most powerful
#the steps will be the same as before

forest_reg = RandomForestRegressor()
forest_reg.fit(features_400_train, label_train)

RandomForestRegressor()

After creating an instance for each regression model, we fit each of them on the training data. 

After each fit we predict on training data and compare the training labels to analyze the accuracy.


In [ ]:
#after training we check out model by predicting using out training data
#we compare out predictions on training data vs. the labels and study the difference
# RMSE is the Root Mean Square Error, which measures the difference between predicted values and the labels. 
# this measure is common and works well with regression models.

train_pred = linreg.predict(features_400_train)
lin_mse = mean_squared_error(label_train, train_pred)
lin_rmse = np.sqrt(lin_mse)
print('linreg RMSE: ', lin_rmse)

linreg RMSE:  18.257244816065786


In [ ]:
#predicting with the Decision Tree Regression 
tree_pred = tree_reg.predict(features_400_train)
tree_mse = mean_squared_error(label_train, tree_pred)
tree_rmse = np.sqrt(tree_mse)
print('decision tree RMSE: ', tree_rmse)

decision tree RMSE:  1.5799208840951517


In [ ]:
#prediction with Forest Regression
forest_pred = forest_reg.predict(features_400_train)
forest_mse = mean_squared_error(label_train, forest_pred)
forest_rmse = np.sqrt(forest_mse)

print('forest RMSE:',  forest_rmse)

forest RMSE: 7.616102596572324


In [ ]:
#here is a small test using a runner with a 60 second 400m.
# the number 1 represents a grade level.
#using this we can run a simple test to see how acurate a single prediction is.
#the array corresponds to 400m time, 9th grade, 10th grade, 11th grade, and 12th grade (in this order).

test_arr = np.array([60,0,0,0,1])
test_arr = test_arr.reshape(1,-1)

#predicting a single result with linreg
test_pred_lin = linreg.predict(test_arr)
test_pred_lin

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([315.65007503])

In [ ]:
#here we run the same test as before, with a single sample, and compare how the decision tree predicts.
#although the decision tree has a lower RMSE than linreg, the prediction is futher off what we would expect.
# this shows that the decision tree likely is overfitting the data

#predicting a single result with Decison tree
test_pred_tree = tree_reg.predict(test_arr)
test_pred_tree

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([314.42])

In [ ]:
# predicting a single result with forest regression 
test_pred_forest = forest_reg.predict(test_arr)
test_pred_forest

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([320.6814])

In [ ]:
# the same cross val is run using linear regression

lin_scores = cross_val_score(linreg, features_400_train, label_train, 
                         scoring='neg_mean_squared_error', cv=10)

#compared to the original linreg, the mean here is marginally better, but it is still insightful to check 
#the original values and compare.

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [19.09563125 29.93328479 12.5296704  13.46648559 15.71803556 11.17044303
 18.25920875  6.17790534 28.4540226  33.28128846]
Mean: 18.808597576533426
Standard Deviation: 8.497213305263687


In [ ]:
#here I import cross_val_score which will allow me to test the decision tree 
#and improve how it fits by taking small chunks of the data and training on each chunk
# this results in a better fitting.

tree_scores = cross_val_score(tree_reg, features_400_train, label_train, 
                         scoring='neg_mean_squared_error', cv=10)

# displaying the scors shows a larger mean
#this is expected because before, the data was being overfit
tree_rmse_scores = np.sqrt(-tree_scores)
display_scores(tree_rmse_scores)

Scores: [18.32477503 16.53779248  8.17879209 18.90600381 29.9600968  15.41251764
 21.36050468 15.97954505 23.50835277 26.81169428]
Mean: 19.49800746133316
Standard Deviation: 5.917620724395623


In [ ]:
#just as with the decision tree, randome forest was likely overfitting the training data
# when using crossval we see that the mean is a amost twice as bad
# still, this model produces the best results
#this is also expected because this model is more powerful 

forest_scores = cross_val_score(forest_reg, features_400_train, label_train, 
                         scoring='neg_mean_squared_error', cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [14.09282139 26.41068888 14.30443963 16.83156233 25.52786379 16.69194257
 13.87241126 11.81376592 20.48705698 31.4001519 ]
Mean: 19.14327046308307
Standard Deviation: 6.22409208886347


In [ ]:
#after going through the models, we import one last package, GridSearchCV
#this sorts through many parameters and returns the best one.

param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]}
]
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

#here the grid search model is used to fit the training data
#next we print out the best params and best estimators
grid_search.fit(features_400_train, label_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'bootstrap': False, 'max_features': 4, 'n_estimators': 10}
RandomForestRegressor(bootstrap=False, max_features=4, n_estimators=10)


In [ ]:
#here the features and estimators are zip together with the scores
#then they are printed out so we can see which returned the best and the range of values 
#that result from the changes in parameters

cv_res = grid_search.cv_results_

for mean_score, params in zip(cv_res['mean_test_score'], cv_res['params']):
  print(np.sqrt(-mean_score), params)

24.79805168770669 {'max_features': 2, 'n_estimators': 3}
24.895044984539037 {'max_features': 2, 'n_estimators': 10}
22.659928656093754 {'max_features': 2, 'n_estimators': 30}
21.458597692150818 {'max_features': 4, 'n_estimators': 3}
22.463226494873787 {'max_features': 4, 'n_estimators': 10}
21.789447113635898 {'max_features': 4, 'n_estimators': 30}
23.60635619781824 {'max_features': 6, 'n_estimators': 3}
21.27430082366777 {'max_features': 6, 'n_estimators': 10}
20.798955460696803 {'max_features': 6, 'n_estimators': 30}
20.51807131094412 {'max_features': 8, 'n_estimators': 3}
20.71170994062162 {'max_features': 8, 'n_estimators': 10}
21.320854344155435 {'max_features': 8, 'n_estimators': 30}
23.051189455933365 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
23.311961659914704 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
20.84844230022183 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
21.287174896793296 {'bootstrap': False, 'max_features': 3, 'n_estim

In [ ]:
#lastly we isolate the best estimators from the model
#we then predict the TEST data and compare to the test labels
final_model = grid_search.best_estimator_
final_pred = final_model.predict(features_400_test)

final_mse = mean_squared_error(label_test, final_pred)
final_rmse = np.sqrt(final_mse)
print(final_model)
print(final_rmse)

32.880637431867484


In [ ]:
test_pred = final_model.predict(test_arr)
print(test_pred)

[314.42]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
